In [ ]:
import sys, os

gameFolder = "DOA++-MAME"
sys.path.append(os.path.join(os.path.abspath(''), '../../games',gameFolder))   

In [ ]:
import random
import numpy as np
from collections import deque
import cv2  # pytype:disable=import-error
cv2.ocl.setUseOpenCL(False)

import gym
from gym import spaces

from diambraMameGym import *

from stable_baselines import logger
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import VecFrameStack

from stable_baselines.common.policies import CnnPolicy
from stable_baselines import PPO2

In [ ]:
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=6):
        """
        Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be last action (env.n_actions[0]*env.n_actions[1] -1).
        :param env: (Gym Environment) the environment to wrap
        :param noop_max: (int) the maximum value of no-ops to run
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = env.n_actions[0]*env.n_actions[1] -1
        print("Noop action N = ", self.noop_action)

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = random.randint(1, self.noop_max + 1)
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, action):
        return self.env.step(action)

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """
        Return only every `skip`-th frame (frameskipping)
        :param env: (Gym Environment) the environment
        :param skip: (int) number of `skip`-th frame
        """
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=env.observation_space.dtype)
        self._skip = skip

    def step(self, action):
        """
        Step the environment with the given action
        Repeat action, sum reward, and max over last observations.
        :param action: ([int] or [float]) the action
        :return: ([int] or [float], [float], [bool], dict) observation, reward, done, information
        """
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        """
        clips the reward to {+1, 0, -1} by its sign.
        :param env: (Gym Environment) the environment
        """
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """
        Bin reward to {+1, 0, -1} by its sign.
        :param reward: (float)
        """
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, hw_obs_resize = [84, 84]):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        :param env: (Gym Environment) the environment
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = hw_obs_resize[1]
        self.height = hw_obs_resize[0]
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.height, self.width, 1),
                                            dtype=env.observation_space.dtype)

    def observation(self, frame):
        """
        returns the current observation from a frame
        :param frame: ([int] or [float]) environment frame
        :return: ([int] or [float]) the observation
        """
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class FrameStack(gym.Wrapper):
    def __init__(self, env, n_frames):
        """Stack n_frames last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        stable_baselines.common.atari_wrappers.LazyFrames
        :param env: (Gym Environment) the environment
        :param n_frames: (int) the number of frames to stack
        """
        gym.Wrapper.__init__(self, env)
        self.n_frames = n_frames
        self.frames = deque([], maxlen=n_frames)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * n_frames),
                                            dtype=env.observation_space.dtype)

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        for _ in range(self.n_frames):
            self.frames.append(obs)
        return self._get_ob()

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.n_frames
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = spaces.Box(low=0, high=1.0, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class LazyFrames(object):
    def __init__(self, frames):
        """
        This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to np.ndarray before being passed to the model.
        :param frames: ([int] or [float]) environment frames
        """
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=2)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]


def make_diambra(diambraGame, diambra_kwargs):
    """
    Create a wrapped atari Environment
    :param env_id: (str) the environment ID
    :return: (Gym Environment) the wrapped atari environment
    """
    env = diambraGame(**diambra_kwargs)
    env = NoopResetEnv(env, noop_max=6)
    #env = MaxAndSkipEnv(env, skip=4)
    return env


def wrap_deepmind(env, clip_rewards=True, frame_stack=1, scale=False):
    """
    Configure environment for DeepMind-style Atari.
    :param env: (Gym Environment) the atari environment
    :param clip_rewards: (bool) wrap the reward clipping wrapper
    :param frame_stack: (bool) wrap the frame stacking wrapper
    :param scale: (bool) wrap the scaling observation wrapper
    :return: (Gym Environment) the wrapped atari environment
    """

    # Resizing observation from H x W x 3 to hw_obs_resize[0] x hw_obs_resize[1] x 1
    env = WarpFrame(env, hw_obs_resize = [84, 84])
    
    # Scales observations normalizing them between 0.0 and 1.0
    if scale:
        env = ScaledFloatFrame(env)
        
    # Clip rewards using sign function
    if clip_rewards:
        env = ClipRewardEnv(env)
        
    # Stack #frame_stack frames together
    if frame_stack > 1:
        env = FrameStack(env, frame_stack)
        
    return env

In [ ]:
def make_diambra_env(env_id, num_env, seed, wrapper_kwargs=None,
                   start_index=0, allow_early_resets=True,
                   start_method=None, use_subprocess=False):
    """
    Create a wrapped, monitored VecEnv for Atari.
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the initial seed for RNG
    :param wrapper_kwargs: (dict) the parameters for wrap_deepmind function
    :param start_index: (int) start rank index
    :param allow_early_resets: (bool) allows early reset of the environment
    :param start_method: (str) method used to start the subprocesses.
        See SubprocVecEnv doc for more information
    :param use_subprocess: (bool) Whether to use `SubprocVecEnv` or `DummyVecEnv` when
        `num_env` > 1, `DummyVecEnv` is usually faster. Default: False
    :return: (VecEnv) The atari environment
    """
    if wrapper_kwargs is None:
        wrapper_kwargs = {}

    def make_env(rank):
        def _thunk():
            env = make_diambra(env_id)
            env.seed(seed + rank)
            env = Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(rank)),
                          allow_early_resets=allow_early_resets)
            return wrap_deepmind(env, **wrapper_kwargs)
        return _thunk
    set_global_seeds(seed)

    # When using one environment, no need to start subprocesses
    if num_env == 1 or not use_subprocess:
        return DummyVecEnv([make_env(i + start_index) for i in range(num_env)])

    return SubprocVecEnv([make_env(i + start_index) for i in range(num_env)],
                         start_method=start_method)

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["frame_ratio"] = 1
diambraKwargs["player_id"] = "P1"
diambraKwargs["character"] = "Random"

env = make_diambra(diambraMame, diambra_kwargs = diambraKwargs)
env = Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(0)),
              allow_early_resets=True)
env = wrap_deepmind(env)

In [ ]:
observation = env.reset()

while True:

    action = random.randint(0, 35)
    
    observation, reward, done, info = env.step(action)

    print("Frames shape:", observation.shape)
    print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    #print("Rewards = ", info["rewards"])
    #print("HealthP1 = ", info["healthP1"])
    #print("HealthP2 = ", info["healthP2"])
    #print("PositionP1 = ", info["positionP1"])
    #print("PositionP2 = ", info["positionP2"])
    #print("WinP1 = ", info["winsP1"])
    #print("WinP2 = ", info["winsP2"])
    print("Done = ", done)

    if done:
        print("Resetting Env")
        observation = env.reset()
        
env.close()

In [ ]:
# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multiprocessing training (num_env=4 => 4 processes)
env = make_diambra_env('PongNoFrameskip-v4', num_env=4, seed=0,  wrapper_kwargs = {"frame_stack": True})

# OR 
#env = make_atari_env('PongNoFrameskip-v4', num_env=1, seed=0)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

model = PPO2(CnnPolicy, env, verbose=1)

In [ ]:
# Train the agent
time_steps = 1000
model.learn(total_timesteps=time_steps)

In [ ]:
# Enjoy trained agent
eval_env = make_atari_env('PongNoFrameskip-v4', num_env=1, seed=0,  wrapper_kwargs = {"frame_stack": True})
obs = eval_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=False)
    obs, reward, done, info = eval_env.step(action)
    eval_env.render()
    if done:
        obs = eval_env.reset()
eval_env.close()